In [1]:
import re
import openpyxl
import pandas as pd

def read_workbook(file_path):
    workbook = openpyxl.load_workbook(file_path)
    formulas = {
        'sheet': [],
        'address': [],
        'value': [],
    }

    for sheet_name in workbook.sheetnames:
        sheet = workbook[sheet_name]
        for row in range(1, sheet.max_row + 1):
            for col in range(1, sheet.max_column + 1):
                cell = sheet.cell(row=row, column=col)
                if cell.value is not None:
                    formulas['sheet'].append(sheet_name.replace(' ', '_'))
                    formulas['address'].append(cell.coordinate)
                    
                    if isinstance(cell.value, openpyxl.worksheet.formula.ArrayFormula):
                        formulas['value'].append(cell.value.text)
                    else:
                        formulas['value'].append(cell.value)

    return pd.DataFrame(formulas)

file_path = 'client_docs/example_data.xlsx'
formula_df = read_workbook(file_path)

In [2]:
def remove_duplicate(list_data):
    return list(dict.fromkeys(list_data).keys())

def extract_excel_addresses(text):
    """ Matches string for excel address
    
    Example: 
        "B5, Z65, Sheet2!C10, and Sheet3!AZ50."
        [('B5', ''),
         ('Z65', ''),
         ('Sheet2!C10', 'Sheet2!'),
         ('Sheet3!AZ50', 'Sheet3!')]
    """
    pattern = r"((\w*!)?[A-Z]{1,3}[0-9]{1,7})"
    matches = re.findall(pattern, text)
    matches = remove_duplicate(matches)
    matches = sorted(matches, key=lambda x: len(x[0]), reverse=True)
    return matches

def extract_excel_array_addr(text):
    """ Matches string for excel address
    
    Example: 
        "B5, Z65, Sheet2!C10, and Sheet3!AZ50."
        [('B5', ''),
         ('Z65', ''),
         ('Sheet2!C10', 'Sheet2!'),
         ('Sheet3!AZ50', 'Sheet3!')]
    """
    pattern = r"\w*![A-Z]{1,3}[0-9]{1,7}(:\w*!([A-Z]{1,3}[0-9]{1,7}))"
    matches = re.findall(pattern, text)
    matches = remove_duplicate(matches)
    return matches

def str_replace(text, old_substring, new_substring):
    """
    Replace occurrences of old_substring with new_substring in the text.

    Parameters:
        text (str): The original text.
        old_substring (str): The substring to be replaced.
        new_substring (str): The new substring to replace old_substring.

    Returns:
        str: The text with replacements made.
    """
    result = ""
    i = 0
    while i < len(text):
        match_substring = text[i:i + len(old_substring)] == old_substring
        excla_not_on_left = '!' not in text[i-3:i-1]
        if match_substring and excla_not_on_left:
            result += new_substring
            i += len(old_substring)
        else:
            result += text[i]
            i += 1
    return result


def process_formula_addr(row):
    formula = row['value']
    sheet_name = row['sheet']
    if isinstance(formula, str) and formula.startswith('='):
        formula = formula.replace('$', '')
        excel_cell_addr = extract_excel_addresses(formula)
        target_text = []
        for search_result in excel_cell_addr:
            addr, sheet = search_result
            if sheet == '':
                target_text.append(addr)
                new_addr = f'{sheet_name}!{addr}'
                formula = str_replace(formula, addr, new_addr)

        excel_array_cell_addr = extract_excel_array_addr(formula)
        for search_result in excel_array_cell_addr:
            replace_me, addr = search_result
            formula = formula.replace(replace_me, ':' + addr)

    return formula


# formula_df[1156:1157].apply(process_formula_addr, axis=1).values
formula_df['formula_addr'] = formula_df.apply(process_formula_addr, axis=1)
formula_df.to_csv('output/formula.csv')